In [55]:
import os
from collections import OrderedDict
import sys
import glob
from subprocess import Popen, PIPE, DEVNULL
from itertools import islice
from multiprocessing import cpu_count as CC

In [41]:
mainDir='/accounts/ekarakoy/DEV-ALL/UNCERTAINTIES/UNCPY/UNC_notebooks'
dataDir='/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/'
testDir = 'Systematic_Error_Sensitivity'
filename = 'test'
ext = '.par'
fpath = os.path.join(mainDir, testDir, filename)

In [79]:
n_vvv = 6
n_nnn = 8
parDict = OrderedDict()
ifn = 'S2003150182636.L1A_GAC'
ofn_base = ifn.split('.')[0]
ofn_ext = '.L2'
ofn = '%s_baseline%s' % (ofn_base, ofn_ext)
parDict['ifile'] = os.path.join(dataDir, 'YeOldeStuffe/L1A', ifn)
parDict['ofile'] = os.path.join(mainDir,testDir, ofn)
parDict['l2prod'] = "Lt_nnn,Rrs_vvv"
parfileName = 'sys_error_baseline.par'
exp = 'baseline' + ext
fpath = os.path.join(mainDir,testDir,exp)
with open(fpath,'w') as fp:
    PrParCont(parDict, fp=fp)

add_lt_noise = '1.0'
lt_noise_scale = ''.join(['['] + ['-1,'] * 7 + ['-1]'])
maxIters = 10
parDict['add_lt_noise'] = add_lt_noise
parDict['lt_noise_scale'] = lt_noise_scale
for i in range(1, maxIters):
    ofn = ofn_base + '_%d' % i + ofn_ext
    parDict['ofile'] = os.path.join(mainDir, testDir, ofn)
    bias = 0.001 + (i - 1) * 0.001
    exp = 'sys_error_%d%s' % (i, ext)
    fpath = os.path.join(mainDir, testDir, exp)
    bias_frac = ''.join(['['] + ['%.3f,' % bias] * 6 + ['0,0]'])
    parDict['bias_frac'] = bias_frac
    with open(fpath, 'w') as fp:
        PrParCont(parDict, fp=fp)

In [38]:
def PrParCont(pdict, msg=None, fp=sys.stdout):
    if msg:
        print(msg)
    for k,v in pdict.items():
        print('%s=%s' %(k,v), file=fp)

In [80]:
parfiles = glob.iglob(os.path.join(mainDir,testDir,'*.par'))

In [66]:
def CommandGen(parfiles):
    for pf in parfiles:
        cmd = 'l2gen par=%s' % pf
        yield cmd

In [67]:
maxWorkers = 2 * CC() - 1
def Runner(tasks, workers=maxWorkers):
    workers = min(maxWorkers,workers)
    processes = (Popen(task, shell=True, stdout=DEVNULL, stderr=PIPE)
                for task in tasks)
    runningProcs = list(islice(processes, workers))
    while runningProcs:
        for i, process in enumerate(runningProcs):
            if process.poll() is not None: # process has finished
                runningProcs[i] = next(processes, None)
                if runningProcs[i] is None:
                    del runningProcs[i]
                    break

In [81]:
tasks = CommandGen(parfiles)

In [82]:
Runner(tasks, workers=5)

no procs. to add to slot 4
no procs. to add to slot 2
no procs. to add to slot 2
no procs. to add to slot 0
no procs. to add to slot 0
